# 延后初始化
框架的延后初始化（defers initialization），即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。

在以后，当使用卷积神经网络时，由于输入维度（即图像的分辨率）将影响每个后续层的维数，有了该技术将更加方便。现在我们在编写代码时无须知道维度是什么就可以设置参数，这种能力可以大大简化定义和修改模型的任务。接下来，我们将更深入地研究初始化机制。
## 实例化网络
首先，让我们实例化一个多层感知机。

In [2]:
import torch
from torch import nn
# 在 PyTorch 中，nn.LazyLinear 是一个延后初始化的线性层，它在第一次前向传播时会自动推断输入特征的大小。
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(),nn.Linear(256,10))
net

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

此时，因为输入维数是未知的，所以网络不可能知道输入层权重的维数。因此，框架尚未初始化任何参数，我们通过尝试访问以下参数进行确认。

In [7]:
[net[i].state_dict() for i in range(len(net))], net[0].weight.data

([OrderedDict([('weight', <UninitializedParameter>),
               ('bias', <UninitializedParameter>)]),
  OrderedDict(),
  OrderedDict([('weight',
                tensor([[ 0.0506, -0.0086, -0.0197,  ...,  0.0607, -0.0364, -0.0078],
                        [-0.0352, -0.0223,  0.0352,  ..., -0.0179,  0.0226, -0.0293],
                        [-0.0120, -0.0024, -0.0057,  ...,  0.0605,  0.0416,  0.0039],
                        ...,
                        [ 0.0319, -0.0461, -0.0398,  ..., -0.0402, -0.0599, -0.0141],
                        [-0.0227, -0.0535, -0.0123,  ..., -0.0523,  0.0532,  0.0570],
                        [-0.0252, -0.0276,  0.0174,  ...,  0.0569,  0.0271,  0.0223]])),
               ('bias',
                tensor([-0.0521,  0.0265, -0.0226,  0.0510, -0.0022, -0.0150, -0.0408, -0.0356,
                        -0.0168,  0.0497]))])],
 tensor([]))

接下来让我们将数据通过网络，最终使框架初始化参数。

In [8]:
low = torch.finfo(torch.float32).min/10
high = torch.finfo(torch.float32).max/10
X = torch.zeros([2,20],dtype=torch.float32).uniform_(low, high)
net(X)
net

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

一旦我们知道输入维数是20，框架可以通过代入值20来识别第一层权重矩阵的形状。识别出第一层的形状后，框架处理第二层，依此类推，直到所有形状都已知为止。注意，在这种情况下，只有第一层需要延迟初始化，但是框架仍是按顺序初始化的。等到知道了所有的参数形状，框架就可以初始化参数。